In [ ]:
# Run this cell to get everything set up.
from lec_utils import *
import lec25_util as util

<div class="alert alert-info" markdown="1">

#### Lecture 25

# Computer Vision, Conclusion

### EECS 398: Practical Data Science, Winter 2025

<small><a style="text-decoration: none" href="https://practicaldsc.org">practicaldsc.org</a> • <a style="text-decoration: none" href="https://github.com/practicaldsc/wn25">github.com/practicaldsc/wn25</a> • 📣 See latest announcements [**here on Ed**](https://edstem.org/us/courses/69737/discussion/5943734) </small>
    
</div>

<script type="text/x-mathjax-config">
 MathJax.Hub.Config({
   TeX: {
     extensions: ["color.js"],
     packages: {"[+]": ["color"]},
   }
 });
 </script>
 <script src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.5/MathJax.js?config=TeX-AMS_HTML"></script>

### Final exam details 🙇

- The Final Exam is on **Monday, April 28th from 10:30AM-12:30PM**.<br><small>You'll receive your assigned room via email by Thursday. If we've made alternate arrangements, I'll confirm them with you on Thursday as well.</small>

- Lectures 13-24, Homeworks 7-11, and Discussions 8-13 are all in scope. The exam is **not cumulative**, but post-midterm material builds upon pre-midterm material.<br><small>For instance, you'll need to know about loss functions and basic `pandas`, since we've used such ideas in the second half of the semester repeatedly.</small> 

- You can bring **two** double-sided handwritten notes sheets.<br><small>Feel free to bring your midterm notes sheet as one of your two sheets!</small>

- We have two review sessions this week:
    - Wednesday 4/23, 3-5PM, 1670 BBB.
    - Saturday 4/26, 1-3PM, 1670 BBB.
    There will be a different worksheet for each review session. We will post both worksheets **today**.
    
    Do them in advance **as if they were mock exams**, and we'll take them up together.

- The [**study site**](https://study.practicaldsc.org) is the most important resource, followed by lecture slides, and then assignments.<br><small>When working through old discussion problems, work as if you're in a real exam – time yourself, don't use the internet, no music, sit somewhere public, etc.</small>

### Agenda 📆

- Computer vision 👾.
- Parting thoughts 💭.

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
Remember that you can always ask questions anonymously at the link above!

## Computer vision 👾

---

We'll wrap up the semester by looking at a fun application area of the tools we've seen – computer vision.<br>None of the <b>new</b> material today is in scope for the exam, but a lot of it is implicitly review.

<center>
    
<img src="imgs/digits-intro.png" width=1000>
    
</center>

<center><b>Goal</b>: Given an image of a digit, <b>classify</b> the digit as either 0, 1, 2, 3, 4, 5, 6, 7, 8, or, 9.</center>

<center>This is a multi-class <b>classification</b> problem with 10 classes.</center>

<center>This is also an example of <b>computer vision</b>, <br>a branch of machine learning that deals with learning patterns in images and videos.</center>

### The MNIST dataset

- The MNIST dataset contains 70,000 **labeled** images of digits, all of which are **28 pixels by 28 pixels** and grayscale (no color).<br><small>MNIST stands for "Modified National Institute of Standards and Technology". Yann LeCun et. al. are responsible for curating the dataset; read the original webpage for it [**here**](https://yann.lecun.com/exdb/mnist/). </small>

<center><img src="imgs/mnist-many.jpg" width=500></center>

- The dataset is pre-split into a training set of 60,000 images and test set of 10,000 images.

- Test set performance on MNIST is a common benchmark for evaluating the quality of a classifier.<br><small>Convolutional neural networks, one of the most popular model architectures for computer vision, were developed specifically in the context of achieving high accuracy on MNIST.<br>There are other, similarly-purposed datasets too, like FashionMNIST.</small>

- State of the art **neural network**-based models have achieved test set accuracies of as high as 99.87\%.<br><small>See the leaderboard [**here**](https://paperswithcode.com/sota/image-classification-on-mnist)!</small>

### Loading the data

- `sklearn` has a built-in function for loading datasets from [OpenML](https://www.openml.org/search?type=data).

In [ ]:
from sklearn.datasets import fetch_openml
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=True)
# The documentation here: https://www.openml.org/search?type=data&status=active&id=554
# tells us that the first 60,000 rows constitute the training set.
X_train, X_test = X.iloc[:60000], X.iloc[60000:]
y_train, y_test = y.iloc[:60000].astype(int), y.iloc[60000:].astype(int)

- What do `X_train` and `y_train` actually look like?

In [ ]:
X_train

In [ ]:
y_train

### From vectors to images

- In `X_train`, each image is represented by a $28 \cdot 28 = 784$-dimensional vector, representing a **flattened** version of the image.

In [ ]:
X_train.iloc[98]

- The first 28 pixels are the first **row** of the image, the second 28 pixels are the second row of the image, and so on. To view the image, we can **reshape** the vector into a 2D grid.

In [ ]:
X_train.iloc[98].to_numpy().reshape((28, 28))

- Each pixel is represented with a value from 0 to 255, where **larger values are more intense** (darker in the plot below).

In [ ]:
# We'll keep using image 98 as an example, so remember that it's a 3!
util.show_image(X_train.iloc[98])

### The distribution of the training set

- Before training any models, we should assess whether there's any class imbalance in the **training set**.<br><small>Remember, we shouldn't peek at the test set until we've actually trained a model.</small>

In [ ]:
y_train.value_counts(normalize=True).sort_index().plot(kind='bar', title='Distribution of Digits in the Training Set')

- The 10 possible digits seem to appear at roughly the same frequency.

### Model #1: $k$-Nearest Neighbors 🏡🏠

- We _can_ use $k$-nearest neighbors to predict the digit in a new image, $\vec{x}_\text{new} \in \mathbb{R}^{784}$.<br>Intuitively, this means finding the $k$ "most similar" images to $\vec{x}_\text{new}$.<br><small>Remember, $k$-nearest neighbors is a **classification** method for **supervised learning**.</small>

- Since we're treating each image in our training set as a "flat" vector in $\mathbb{R}^{784}$, we're ignoring any **spatial** patterns in each image.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
model_knn = KNeighborsClassifier(n_neighbors=100) # Arbitrary choice; remember, there are 60,000 points in the training set.
model_knn.fit(X_train, y_train)

- Note that "training" a $k$-NN classifier (i.e. calling `fit`) is instant, because nearest neighbor models do all of their calculation upon calling `predict`.

- Calling `predict` is much slower than calling `fit` – for each input to `predict`, the model must find the distance between the input vector and all 60,000 vectors in the training set to see which $k=100$ are the closest.

In [ ]:
model_knn.predict(X_train.iloc[[98]])

In [ ]:
# Accuracy on test set. Takes ~10 seconds on my computer, but is fairly high.
y_test_pred = model_knn.predict(X_test)
(y_test == y_test_pred).mean()

### What kinds of errors does the model make?

- A 100-nearest neighbor classifier achieves 94.4\% test set accuracy.

- To further understand the classifier's performance, we can draw its **confusion matrix**.

In [ ]:
X_test_labeled = X_test.assign(
    true=y_test,
    pred=y_test_pred
)
util.show_confusion(y_test, y_test_pred, title='Confusion Matrix for 100-Nearest Neighbors Model')

- Some of the most common errors seem to be:
    - Predicting 1 when the true digit is 2 or 7.
    - Predicting 7 when the true digit is 2.
    - Predicting 9 when the true digit is 4 or 7.

- Let's peek at some of those cases!

### Examining misclassified images

- Run the cell below repeatedly to see a randomly-chosen image from the test set that we classified incorrectly.

In [ ]:
util.show_image_and_label(X_test_labeled.query('pred != true').sample().iloc[0])

- Run the cell below repeatedly to see a randomly-chosen image from the test set that **we incorrectly classified as a 1**.

In [ ]:
util.show_image_and_label(X_test_labeled.query('pred != true and pred == 1').sample().iloc[0])

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
Remember that you can always ask questions anonymously at the link above!

### Downsides of $k$-nearest neighbors

- In the example below, our 100-nearest neighbor classifier predicted 1, when the true label was 2.

In [ ]:
util.show_image_and_label(X_test_labeled.query('pred != true and pred == 1').iloc[1])

- One downside: $k$-nearest neighbors doesn't incorporate any **probability** in its decision-making process.<br>What if we could get a probability that the above image is of a 0, 1, 2, 3, ..., 9?

- Another downside: Classifying a new image takes relatively long.<br>What if we could learn patterns from the training set in advance, so that predicting new images is relatively quick?

### Model #2: Multinomial logistic regression 📈

- Multinomial logistic regression, or softmax regression, predicts the probability that an image $\vec x_i \in \mathbb{R}^{784}$ belongs to each class.

$$P(\text{image } \vec x_i \text{ is of digit }j) = P(y_i = j | \vec{x}_i) = \frac{e^{\vec{w}_j \cdot \text{Aug}(\vec{x}_i)}}{\sum_{k=0}^9 e^{\vec w_k \cdot \text{Aug}(\vec x_i)}}$$

<center><small>Here, $j$ could be 0, 1, 2, ..., 9.</small></center>

- To train a multinomial logistic regression model for this task, we'll ultimately need to find 10 optimal parameter vectors, $\vec w_0^*$, $\vec w_1^*$, ..., $\vec w_9^*$.

### Multinomial logistic regression in `sklearn`

- As we've seen before, the `LogisticRegression` class supports multinomial logistic regression.

In [ ]:
from sklearn.linear_model import LogisticRegression
model_log = LogisticRegression(multi_class='multinomial', penalty='l1', solver='saga')
model_log

- Given that we have 60,000 training examples, each of which are 784-dimensional, training on the full training set takes quite a while – longer than we have time to run in lecture! Just to demonstrate, we'll fit on just the first 10,000 rows of the training set.

In [ ]:
model_log.fit(X_train.head(10000), y_train.head(10000))

- While calling `fit` takes a while, calling `predict` is fast.

In [ ]:
util.show_image(X_train.iloc[[98]])

In [ ]:
model_log.predict(X_train.iloc[[98]])

In [ ]:
# MUCH faster than with the k-nearest neighbors model.
model_log.score(X_test, y_test)

### What kinds of errors does _this_ model make?

- The our multinomial logistic regression model, trained on just the first 10,000 rows of the training set, achieves 90.12\% test set accuracy.

- Let's peek at the confusion matrix once again.

In [ ]:
X_test_labeled = X_test.assign(
    true=y_test,
    pred=model_log.predict(X_test)
)
util.show_confusion(y_test, model_log.predict(X_test), title='Confusion Matrix for Multinomial Logistic Regression Model')

- The most common types of errors are now different! Common errors:
    - Predicting 8 when the true digit is a 2 or 5.
    - Predicting 3 when the true digit is a 5 (or vice versa).

### Modeling uncertainty

- Let's look at test set images that the multinomial logistic regression model misclassified.

In [ ]:
util.show_image_and_label(X_test_labeled.query('pred != true and pred == 8').iloc[15])

- We can use `predict_proba` to see the distribution of predicted probabilities per class.

In [ ]:
model_log.predict_proba(X_test_labeled.query('pred != true and pred == 8').iloc[[15], :-2])

In [ ]:
util.visualize_probs(model_log.predict_proba(X_test_labeled.query('pred != true and pred == 8').iloc[[15], :-2]))

### Other close calls

- Repeatedly run the cell below to look at the distribution of predicted probabilities for misclassified examples.

In [ ]:
t = X_test_labeled.query('pred != true').reset_index(drop=True)
t = t.assign(second_highest_prob = pd.DataFrame(model_log.predict_proba(t.iloc[:, :-2])).apply(lambda r: r.sort_values().iloc[-2], axis=1))
p = t[t['second_highest_prob'] >= 0.3].sample().iloc[0]
util.show_image_and_label(p.iloc[:-1].astype(int)).show()
util.visualize_probs(model_log.predict_proba(p.iloc[:-3].to_frame().T))

### Visualizing coefficients

- Since there are 10 classes, `model_log` has 10 parameter vectors (each in $\mathbb{R}^{784}$) – one per class.

In [ ]:
model_log.coef_

In [ ]:
model_log.coef_.shape

- We can visualize these coefficients, too!<br>Below, pixels in <b><span style="color:blue">blue</span></b> had a positive coefficient, i.e. increase the probability of class 0. Pixels in <b><span style="color:red">red</span></b> had a negative coefficient, i.e. decrease the probability of class 0.</small>

In [ ]:
px.imshow(model_log.coef_[0].reshape((28, 28)), color_continuous_scale='Rdbu', title='Class 0 Coefficients')

In [ ]:
util.plot_model_coefficients(model_log.coef_)

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
Remember that you can always ask questions anonymously at the link above!

### Reflection

- We've fit two models to the MNIST training set so far:
    - $k$-nearest neighbors.
    - Multinomial logistic regression.

- Both models achieved a test set accuracy above 90\%.

- Both models are _slow_ in some sense:
    - $k$-nearest neighbors is slow at classifying new images.
    - Multinomial logistic regression is slow to train.

- These issues, in part, stem from the fact that our design matrix has $28 \cdot 28 = 784$ columns, i.e. 784 features:

In [ ]:
X_train

- Is there a way we could **reduce** the number of features we use, i.e. reduce the number of columns in the design matrix, and still achieve decent test set performance?

### Principal component analysis (PCA)

- Principal component analysis (PCA) is an **unsupervised learning** technique used for **dimensionality reduction**.

- It'll allow us to take:
    - `X_train`, which has 60,000 rows and 784 columns, and transform it into
    - `X_train_approx`, which has 60,000 rows and $p$ columns, where $p$ is as small as we want (e.g. $p = 2$).

- It creates $p$ **new features**, each of which is a linear combination of all existing 784 features.


    $$\text{new feature 1} = 0.05 \cdot \text{pixel 1} + 0.93 \cdot \text{pixel 2} + ... - \: 0.35 \cdot \text{pixel 784}$$

    $$\text{new feature 2} = - 0.06 \cdot \text{pixel 1} + 0.5 \cdot \text{pixel 2} + ... + \: 0.04 \cdot \text{pixel 784}$$

    $$...$$


    These new features are chosen to capture as much variability (information) in the original data as possible.

- How? The details are out of scope for us, but it leverages the **singular value decomposition** from linear algebra:

$$X = U \Sigma V^T$$

### PCA in `sklearn`

- `sklearn` has an implementation of PCA, which operates like a **transformer**.<br><small>Remember, PCA is an **unsupervised** technique! We don't use the actual digit labels for each image when computing this transformation.</small>

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
pca.fit(X_train)

- Once `fit`, `pca` can transform `X_train` into a **2-column matrix** in a way that retains the bulk of the information:

$$\mathbb{R}^{60000 \times 784} \rightarrow \mathbb{R}^{60000 \times 2}$$

In [ ]:
X_train_approx = pca.transform(X_train)
X_train_approx.shape

In [ ]:
X_train_approx

- When each data point was 784-dimensional, we couldn't visualize our training set.<br>But now, each data point is 2-dimensional, which we can easily visualize with a scatter plot!

### Visualizing principal components

- The new features that PCA creates are called **principal components (PCs)**.<br><small>Note that the principal component values no longer correspond to pixel intensities, which used to range between 0 and 255.</small>

- Plotting PC 2 vs. PC 1 doesn't lead to a ton of insight:

In [ ]:
util.show_2_pcs(X_train_approx, y_train)

### Clusters in principal components

- But what if we color each point by its true class?

In [ ]:
util.show_2_pcs(X_train_approx, y_train, color=True)

- **Key idea**: Even when projected onto just two principal components, the 0s tend to look alike, the 1s tend to look alike, and so on!

- This doesn't always happen when using PCA; use it as part of your exploratory data analysis toolkit.

### PCA as a preprocessing step

- We can use PCA as part of a larger modeling pipeline. We've chosen a number of principal components to use in advance, but in practice we should cross-validate.

$$\text{raw data in }\mathbb{R}^{60000 \times 784} \rightarrow \text{transformed data in }\mathbb{R}^{60000 \times 30} \rightarrow \text{predicted probabilities}$$

In [ ]:
from sklearn.pipeline import make_pipeline
model_pca_log = make_pipeline(
    PCA(n_components=30),
    LogisticRegression(multi_class='multinomial', penalty='l1', solver='saga')
)
model_pca_log

- The transformed data is of a much lower dimension than the raw data. As a result, **we can train – and predict – on the full training set very quickly**!

In [ ]:
model_pca_log.fit(X_train, y_train)

- And the test set accuracy is still good!

In [ ]:
model_pca_log.score(X_test, y_test)

## Parting thoughts 👋

---

### You've accomplished a lot!

- You learned **a lot** this semester – you're now among **the most qualified data scientists in the world**!

- You're now **able to start with raw data and come up with accurate, meaningful insights that you can share with others**.

- You know how to use industry-standard data manipulation tools, and you understand the inner-workings of complicated machine learning models.

- You're well-prepared for internships and data science interviews, ready to create your own portfolio of personal projects, and have the background and maturity to succeed in more advanced data science-adjacent courses.

- Data science is a relatively new, rapidly evolving field, **so you'll need to keep evolving with it**.

- The tools of the trade may change, but the core principles won't – fortunately, you now have a strong foundation on which you can develop new skills.

### Future of Practical Data Science

- I've had a blast teaching this class this past year (and will be teaching it again this spring, starting on May 6th).

- Next year, I'll be focusing on my new EECS 298: Mathematics for Machine Learning course, and won't be teaching Practical Data Science.

- I'm hoping to offer Practical Data Science under a permanent course number the following year.

<center><img src="imgs/transcript.png" width=1000><br>My freshman year transcript.</center>

### Thank you, and keep in touch!

- **Thank you** for signing up for this brand-new class!

- The course would not have been possible without our IAs and readers: Abhinav Attaluri, Hana Ahmed, Caleb Hyun, Angela Li, Pranavi Pratapa, Akanksha Rai, Aditya Ashok, In Lorthongpanich, and Jingrui Zhuang.

- Don't be a stranger – our contact information is at [**practicaldsc.org/staff**](https://practicaldsc.org/staff). We want to hear about what you do after this class.<br><small>This semester's course website will remain online permanently at [practicaldsc.org/wn25](https://practicaldsc.org/wn25), so you can refer back to the content.</small>